In [31]:
# if open from google colab, run this command
from google.colab import drive
drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Github/sales-prediction-gru/ipynb
%cd /content/drive/Shareddrives/BUAT\ DIEGO/Data-retail-2015-2022-keluar/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/Shareddrives/BUAT DIEGO/Data-retail-2015-2022-keluar


In [32]:
import os, warnings, random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model, callbacks

import plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

# Set seeds to make the experiment more reproducible.
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [33]:
# df2015 = pd.read_csv('./data-2015-renew.csv')
df2016 = pd.read_csv('./data-2016-renew.csv', parse_dates=['tgl'])
df2017 = pd.read_csv('./data-2017-renew.csv', parse_dates=['tgl'])
df2018 = pd.read_csv('./data-2018-renew.csv', parse_dates=['tgl'])
df2019 = pd.read_csv('./data-2019-renew.csv', parse_dates=['tgl'])
# df2020 = pd.read_csv('./data-2020-renew.csv')
# df2021 = pd.read_csv('./data-2021-renew.csv')
# df2022 = pd.read_csv('./data-2022-renew.csv')

In [34]:
df2016 = df2016.drop(['notxn', 'nonota', 'namabarang', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'kodecustomer', 'iddistributor', 'idpromo', 'iddivisi', 'jenis', 'kategori', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2017 = df2017.drop(['notxn', 'nonota', 'namabarang', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'kodecustomer', 'iddistributor', 'idpromo', 'iddivisi', 'jenis', 'kategori', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2018 = df2018.drop(['notxn', 'nonota', 'namabarang', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'kodecustomer', 'iddistributor', 'idpromo', 'iddivisi', 'jenis', 'kategori', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)
df2019 = df2019.drop(['notxn', 'nonota', 'namabarang', 'hargajual', 'hargabeli', 'diskon', 'hargaafterdiskon', 'subtotal', 'kodeop', 'isbkp', 'kodecustomer', 'iddistributor', 'idpromo', 'iddivisi', 'jenis', 'kategori', 'kodedepartemen', 'departemen', 'namaop', 'kodedivisibarang', 'divisibarang'], axis=1)

#DF 2016

In [35]:
df2016['month_yr'] = df2016['tgl'].apply(lambda x: x.strftime('%m-%y')).astype(str)

In [36]:
df2016['month_yr'] = df2016['month_yr'].map({'01-16':0,'02-16':1,'03-16':2,'04-16':3,'05-16':4,'06-16':5,'07-16':6,'08-16':7,'09-16':8,'10-16':9,'11-16':10,'12-16':11})

In [37]:
train_monthly_2016 = df2016[['tgl', 'month_yr', 'kodejenis', 'barcode', 'qty']]
train_monthly_2016 = train_monthly_2016.sort_values('tgl').groupby(['month_yr', 'kodejenis', 'barcode'], as_index=False)
train_monthly_2016 = train_monthly_2016.agg({'qty':['sum']})
train_monthly_2016.columns = ['month_yr', 'kodejenis', 'barcode', 'qty_cnt']
train_monthly_2016 = train_monthly_2016.query('qty_cnt >= 0 and qty_cnt <= 50')

train_monthly_2016['qty_cnt_month'] = train_monthly_2016.sort_values('month_yr').groupby(['barcode', 'kodejenis'])['qty_cnt'].shift(-1)

display(train_monthly_2016.head())

,month_yr,kodejenis,barcode,qty_cnt,qty_cnt_month
0,0,10101,4902430600835,1.00,4.00
1,0,10101,4902430600859,2.00,2.00
2,0,10101,4902430601023,2.00,1.00
3,0,10101,4902430601191,4.00,6.00
4,0,10101,4902430601207,8.00,4.00


In [38]:
monthly_series_2016 = train_monthly_2016.pivot_table(index=['barcode', 'kodejenis'], columns='month_yr',values='qty_cnt', fill_value=0).reset_index()
monthly_series_2016.head()

month_yr,barcode,kodejenis,0,1,2,3,4,5,6,7,8,9,10,11
0,0000003624241,JB000034,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00
1,00100809,20604,16.78,19.53,26.94,20.63,17.41,16.61,12.52,13.85,42.35,12.77,6.13,9.22
2,00113731,20607,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,25.00,0.75,0.95,1.03
3,00113731,JB000067,12.03,4.82,6.13,4.16,4.84,4.40,6.23,6.69,4.05,0.00,0.00,0.00
4,00125789,20602,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,14.97,0.00,24.96


In [39]:
first_month_2016 = 0
last_month_2016 = 11
serie_size_2016 = 12
data_series_2016 = []

for index, row in monthly_series_2016.iterrows():
    for month1 in range((last_month_2016 - (first_month_2016 + serie_size_2016)) + 1):
        serie = [row['barcode'], row['kodejenis']]
        for month2 in range(serie_size_2016 + 1):
            serie.append(row[month1 + first_month_2016 + month2])
        data_series_2016.append(serie)

columns = ['barcode', 'kodejenis']
[columns.append(i) for i in range(serie_size_2016)]
columns.append('label')

data_series_2016 = pd.DataFrame(data_series_2016, columns=columns)
data_series_2016.head()

In [40]:
data_series_2016 = data_series_2016.drop(['barcode', 'kodejenis'], axis=1)
# monthly_series_2016 = monthly_series_2016.drop(['barcode', 'kodejenis'], axis=1)

In [41]:
labels_2016 = data_series_2016['label']
data_series_2016.drop('label', axis=1, inplace=True)
train_2016, valid_2016, Y_train_2016, Y_valid_2016 = train_test_split(data_series_2016, labels_2016.values, test_size=0.10, random_state=0)
# train_2016, valid_2016, Y_train_2016, Y_valid_2016 = train_test_split(monthly_series_2016, test_size=0.10, random_state=0)

ValueError: ignored

In [ ]:
print("Train set", train_2016.shape)
print("Validation set", valid_2016.shape)
train_2016.head()

In [ ]:
X_train_2016 = train_2016.values.reshape((train_2016.shape[0], train_2016.shape[1], 1))
X_valid_2016 = valid_2016.values.reshape((valid_2016.shape[0], valid_2016.shape[1], 1))

print("Train set reshaped", X_train_2016.shape)
print("Validation set reshaped", X_valid_2016.shape)

In [ ]:
serie_size_2016 =  X_train_2016.shape[1] # 12
n_features_2016 =  X_train_2016.shape[2] # 1

epochs = 1000
batch = 64
lr = 0.001

gru_model_2016 = Sequential()
gru_model_2016.add(L.GRU(10, input_shape=(serie_size_2016, n_features_2016), return_sequences=True))
gru_model_2016.add(L.GRU(6, activation='relu', return_sequences=True))
gru_model_2016.add(L.GRU(1, activation='relu'))
gru_model_2016.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
gru_model_2016.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
gru_model_2016.add(L.Dense(1))
gru_model_2016.summary()

adam = optimizers.Adam(lr)
gru_model_2016.compile(loss='mse', optimizer=adam)

In [ ]:
early_stop = callbacks.EarlyStopping(patience = 10)
gru_history_2016 = gru_model_2016.fit(X_train_2016, Y_train_2016, 
                              validation_data=(X_valid_2016, Y_valid_2016), 
                              batch_size=batch, 
                              epochs=epochs, 
                              verbose=2,
                              callbacks=[early_stop])

In [ ]:
fig, ax1 = plt.subplots(1, sharey=True,figsize=(22,7))

ax1.plot(gru_history_2016.history['loss'], label='Train loss')
ax1.plot(gru_history_2016.history['val_loss'], label='Validation loss')
ax1.legend(loc='best')
ax1.set_title('Regular GRU 2016')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('MSE')

plt.show()

In [ ]:
lstm_train_pred_2016 = gru_model_2016.predict(X_train_2016)
lstm_val_pred_2016 = gru_model_2016.predict(X_valid_2016)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train_2016, lstm_train_pred_2016)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid_2016, lstm_val_pred_2016)))